# AI-LAB SESSION 3: Markov Decision Process

In the third session we will work on the Markov decision process (MDP)

## Lava environments

The environments used are **LavaFloor** (visible in the figure) and its variations.
![LavaFloor](images/lava.png)
The agent starts in cell $(0, 0)$ and has to reach the treasure in $(2, 3)$. In addition to the walls of the previous environments, the floor is covered with lava, there is a black pit of death. What a nice place to visit!

Moreover, the agent can't comfortably perform its actions that instead have a stochastic outcome (visible in the figure)
![DynAct](images/dynact.png)
The action dynamics is the following:
- $P(0.8)$ of moving in the desired direction
- $P(0.1)$ of moving in a direction 90° with respect to the desired direction

Finally, since the floor is covered in lava, the agent receives a negative reward for each of its steps!
- <span style="color:orange">-0.04</span> for each lava cell (L)
- <span style="color:red">-5</span> for the black pit (P). End of episode
- <span style="color:green">+1</span> for the treasure (G). End of episode

## Assignment 1

Your first assignment is to implement the *Value Iteration* algorithm on **LavaFloor**. The solution returned by your algorithm must be a 1-d array of action identifiers where the $i$-th action refers to the $i$-th state. After the correctness of your implementations have been assessed, you can run the algorithms on other four lava environments: **VeryBadLavaFloor**, **NiceLavaFloor**, **BiggerLavaFloor**, **HugeLavaFloor**

### Hint

The first version of your implementation should use for loops and such as in the pseudocode. Once the result is satisfying try to use as many numpy operations as possible to speed up the code

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym
import envs
import numpy as np
from utils.funcs import run_episode, plot
from timeit import default_timer as timer
from tqdm import tqdm_notebook as tqdm

The following function has to be implemented

In [3]:
def value_iteration(environment, maxiters, gamma, delta):
    """
    Performs the value iteration algorithm for a specific environment
    
    Args:
        environment: OpenAI Gym environment
        maxiters: max iterations allowed
        gamma: gamma value
        delta: delta value
        
    Returns:
        policy: 1-d dimensional array of action identifiers where index `i` corresponds to state id `i`
    """
    # v = np.zeros(environment.observation_space.n)#, dtype="double")  # Initial policy
    # iter = 0
    # while True:
    #     v1 = v.copy()
    #     iter += 1
    #     for current_state in range(environment.observation_space.n):
    #         sum_list = [0 for a in environment.actions.keys()]
    #         for a in environment.actions.keys():
    #             for next_state in range(environment.observation_space.n):
    #                 sum_list[a] += environment.T[current_state, a, next_state] \
    #                                * (environment.R[current_state, a, next_state] + gamma*v[next_state])
    #         # print(sum_list)
    #         v[current_state] = np.max(sum_list)
    #     if np.max(np.abs(v-v1)) < delta or iter == maxiters: 
    #         break
    # 
    # pi = np.zeros(environment.observation_space.n, dtype="int8")
    # for current_state in range(environment.observation_space.n):
    #     sum_list = [0 for a in environment.actions.keys()]
    #     for a in environment.actions.keys():
    #         for next_state in range(environment.observation_space.n):
    #                 sum_list[a] += environment.T[current_state, a, next_state] \
    #                                    * (environment.R[current_state, a, next_state] + gamma*v[next_state])
    #     pi[current_state] = np.argmax(sum_list)
    # return np.asarray(pi)
     
    v = np.zeros(environment.observation_space.n, dtype="int8")
    i = 0
    tr = environment.T * environment.R
    trm = np.sum(tr, axis=2)
    while True:
        v1 = np.copy(v)
        i += 1
        v = np.max(trm + np.sum(environment.T*gamma*v, axis=2), axis=1)
        if i == maxiters or np.max(np.abs(v - v1)) < delta:
            break

    p = np.argmax(trm + np.sum(environment.T*gamma*v, axis=2), axis=1)
    return np.asarray(p)


The following code executes and *Value Iteration* and prints the resulting policy

In [4]:
# Learning parameters
delta = 1e-3
gamma = .9
maxiters = 50  # Max number of iterations to perform

envname_list = ["LavaFloor-v0", "VeryBadLavaFloor-v0", "NiceLavaFloor-v0"]

for envname in envname_list:
    print("\n----------------------------------------------------------------")
    print("\tEnvironment: ", envname)
    print("----------------------------------------------------------------\n")

    env = gym.make(envname)
    env.render()

    t = timer()
    policy = value_iteration(env, maxiters, gamma, delta)

    print("\n\nValue Iteration:\n----------------------------------------------------------------"
          "\nExecution time: {0}s\nPolicy:\n{1}".format(round(timer() - t, 4), np.vectorize(env.actions.get)(policy.reshape(
                                                                                            env.rows, env.cols))))


----------------------------------------------------------------
	Environment:  LavaFloor-v0
----------------------------------------------------------------

[['S' 'L' 'L' 'L']
 ['L' 'W' 'L' 'P']
 ['L' 'L' 'L' 'G']]


Value Iteration:
----------------------------------------------------------------
Execution time: 0.0079s
Policy:
[['D' 'L' 'L' 'U']
 ['D' 'L' 'L' 'L']
 ['R' 'R' 'R' 'L']]

----------------------------------------------------------------
	Environment:  VeryBadLavaFloor-v0
----------------------------------------------------------------

[['S' 'L' 'L' 'L']
 ['L' 'W' 'L' 'P']
 ['L' 'L' 'L' 'G']]


Value Iteration:
----------------------------------------------------------------
Execution time: 0.0041s
Policy:
[['R' 'R' 'R' 'D']
 ['D' 'L' 'R' 'L']
 ['R' 'R' 'R' 'L']]

----------------------------------------------------------------
	Environment:  NiceLavaFloor-v0
----------------------------------------------------------------

[['S' 'L' 'L' 'L']
 ['L' 'W' 'L' 'P']
 ['L' '

Correct results for *Value Iteration* can be found [here](results/value_iteration_results.txt)

## Assignment 2

Your first assignment is to implement the *Policy Iteration* algorithm on **LavaFloor**. The solution returned by your algorithm must be a 1-d array of action identifiers where the $i$-th action refers to the $i$-th state. After the correctness of your implementations have been assessed, you can run the algorithms on other four lava environments: **VeryBadLavaFloor**, **NiceLavaFloor**, **BiggerLavaFloor**, **HugeLavaFloor**

### Hint

The first version of your implementation should use for loops and such as in the pseudocode. Once the result is satisfying try to use as many numpy operations as possible to speed up the code.

The following function has to be implemented

In [134]:

def policy_iteration(environment, pmaxiters, vmaxiters, gamma, delta):
    """
    Performs the policy iteration algorithm for a specific environment
    
    Args:
        environment: environment
        pmaxiters: max iterations allowed for the policy improvement
        vmaxiters: max iterations allowed for the policy evaluation
        gamma: gamma value
        delta: delta value
    
    Returns:
        policy: 1-d dimensional array of action identifiers where index `i` corresponds to state id `i`
    """
    # v = np.zeros(env.observation_space.n, dtype="double")  # Initial policy
    # p = np.zeros(env.observation_space.n, dtype="int8")
    # piter = 0
    # while True:
    #     p1 = np.copy(p)
    #     piter += 1
    #     viter = 0
    #     # ---- Evaluate Policy
    #     while True:
    #         v1 = v.copy()
    #         viter += 1
    #         for s in range(env.observation_space.n): #itero sugli stati (for each s in S:)
    #             sum_ = 0
    #             # T(s, pi_s, s')(R(s, pi_s, s') + γV(s'))
    #             for s1 in range(env.observation_space.n): #(for each s' in S:)
    #                 sum_ += env.T[s, p[s], s1]*(env.R[s, p[s], s1] + gamma*v[s1]) 
    #             v[s] = sum_
    #         if np.max(np.abs(v - v1)) < delta or viter == vmaxiters:
    #             break
    #     # ---- Improve Policy
    #     for s in range(env.observation_space.n): #itero sugli stati (for each s in S:)
    #         sum_list = []
    #         for a in range(env.action_space.n): #(for each a in As:)
    #             sum_ = 0
    #             for s1 in range(env.observation_space.n): #(for each s' in S:)
    #                 sum_ += env.T[s, a, s1]*(env.R[s, a, s1] + gamma*v[s1]) #T(s, a, s')(R(s, a, s') + γV(s'))
    #             sum_list.append(sum_)
    #         p[s] = np.argmax(sum_list)
    #     if np.array_equal(p,p1) or piter == pmaxiters:
    #         break
    # 
    # return np.asarray(p)
    p = np.zeros(environment.observation_space.n, dtype="int8")  # Initial policy
    v = np.zeros(environment.observation_space.n, dtype="int8")
    piter = 0
    states = environment.observation_space.n
    tr = environment.T * environment.R
    trm = np.sum(tr, axis=2)
    
    while True:
        p1 = np.copy(p)
        piter += 1
        viter = 0
        while True:
            v1 = np.copy(v)
            viter += 1
            v = (trm + np.sum(environment.T*gamma*v, axis=2))[np.arange(states), p]
            if viter == vmaxiters or np.max(np.abs(v - v1)) < delta:
                break
        p = np.argmax(trm + np.sum(environment.T*gamma*v, axis=2), axis=1)
        if piter == pmaxiters or np.array_equal(p, p1):
            break
    return np.asarray(p)


The following code executes and *Value Iteration* and prints the resulting policy

In [135]:
# Learning parameters
delta = 1e-3
gamma = .9
pmaxiters = 50  # Max number of policy improvements to perform
vmaxiters = 5  # Max number of iterations to perform while evaluating a policy

envname_list = ["LavaFloor-v0", "VeryBadLavaFloor-v0", "NiceLavaFloor-v0"]

for envname in envname_list:
    print("\n----------------------------------------------------------------")
    print("\tEnvironment: ", envname)
    print("----------------------------------------------------------------\n")

    env = gym.make(envname)
    env.render()

    t = timer()
    policy = policy_iteration(env, pmaxiters, vmaxiters, gamma, delta)

    print("\n\nPolicy Iteration:\n----------------------------------------------------------------"
          "\nExecution time: {0}s\nPolicy:\n{1}".format(round(timer() - t, 4), np.vectorize(env.actions.get)(policy.reshape(
                                                                                            env.rows, env.cols))))


----------------------------------------------------------------
	Environment:  LavaFloor-v0
----------------------------------------------------------------

[['S' 'L' 'L' 'L']
 ['L' 'W' 'L' 'P']
 ['L' 'L' 'L' 'G']]


Policy Iteration:
----------------------------------------------------------------
Execution time: 0.0071s
Policy:
[['D' 'L' 'L' 'U']
 ['D' 'L' 'L' 'L']
 ['R' 'R' 'R' 'L']]

----------------------------------------------------------------
	Environment:  VeryBadLavaFloor-v0
----------------------------------------------------------------

[['S' 'L' 'L' 'L']
 ['L' 'W' 'L' 'P']
 ['L' 'L' 'L' 'G']]


Policy Iteration:
----------------------------------------------------------------
Execution time: 0.0052s
Policy:
[['R' 'R' 'R' 'D']
 ['D' 'L' 'R' 'L']
 ['R' 'R' 'R' 'L']]

----------------------------------------------------------------
	Environment:  NiceLavaFloor-v0
----------------------------------------------------------------

[['S' 'L' 'L' 'L']
 ['L' 'W' 'L' 'P']
 ['L'

Correct results for *Policy Iteration* can be found [here](results/policy_iteration_results.txt)

## Comparison

The following code performs a comparison between *Value Iteration* and *Policy Iteration* by plotting the accumulated rewards of each episode with iterations in range $[1, 50]$. Try it on all the previous environments and also on **HugeLavaFloor** (might take a long time if not optimizied via numpy)

In [10]:
# Learning parameters
delta = 1e-3
gamma = 0.9
maxiters = 50  # Max number of iterations to perform

envname = "HugeLavaFloor-v0"

print("\n----------------------------------------------------------------")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

env = gym.make(envname)
env.render()
print("\n")

series = []  # Series of learning rates to plot
liters = np.arange(maxiters + 1)  # Learning iteration values
liters[0] = 1
elimit = 100  # Limit of steps per episode
rep = 10  # Number of repetitions per iteration value
virewards = np.zeros(len(liters))  # Rewards array
c = 0

t = timer()

# Value iteration
for i in tqdm(liters, desc="Value Iteration", leave=True):
    reprew = 0
    policy = value_iteration(env, i, gamma, delta)  # Compute policy
    # Repeat multiple times and compute mean reward
    for _ in range(rep):
        reprew += run_episode(env, policy, elimit)  # Execute policy
    virewards[c] = reprew / rep
    c += 1
series.append({"x": liters, "y": virewards, "ls": "-", "label": "Value Iteration"})

vmaxiters = 5  # Max number of iterations to perform while evaluating a policy
pirewards = np.zeros(len(liters))  # Rewards array
c = 0

# Policy iteration
for i in tqdm(liters, desc="Policy Iteration", leave=True):
    reprew = 0
    policy = policy_iteration(env, i, vmaxiters, gamma, delta)  # Compute policy
    # Repeat multiple times and compute mean reward
    for _ in range(rep):
        reprew += run_episode(env, policy, elimit)  # Execute policy
    pirewards[c] = reprew / rep
    c += 1
series.append({"x": liters, "y": pirewards, "ls": "-", "label": "Policy Iteration"})

print("Execution time: {0}s".format(round(timer() - t, 4)))
np.set_printoptions(linewidth=10000)
print("Leaning rate comparison:\nVI: {0}\nPI: {1}".format(virewards, pirewards))

plot(series, "Learning Rate", "Iterations", "Reward")


----------------------------------------------------------------
	Environment:  HugeLavaFloor-v0
----------------------------------------------------------------

[['S' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L']
 ['L' 'L' 'L' 'L' 'L' 'P' 'L' 'L' 'L' 'L']
 ['L' 'L' 'L' 'W' 'L' 'L' 'W' 'L' 'L' 'L']
 ['L' 'L' 'P' 'W' 'L' 'L' 'W' 'L' 'P' 'L']
 ['L' 'L' 'L' 'W' 'L' 'L' 'W' 'L' 'L' 'L']
 ['L' 'L' 'L' 'W' 'W' 'W' 'W' 'L' 'L' 'L']
 ['L' 'L' 'P' 'L' 'L' 'L' 'L' 'L' 'L' 'P']
 ['L' 'L' 'L' 'L' 'L' 'P' 'L' 'L' 'L' 'L']
 ['L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L']
 ['P' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'G']]




Correct results for comparison can be found here below. Notice that since the executions are stochastic the charts could differ: the important thhing is the global trend.

### BiggerLavaFloor
![Bigger](results/bigger.png)

### HugeLavaFloor
![Huge](results/huge.png)